<a href="https://colab.research.google.com/github/YS-JEOUNG/amazon-review-project/blob/main/code/category_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 기본
import pandas as pd
import numpy as np

# 파일 오픈
import gzip
import json

# datetime
from datetime import datetime

# Warnings       ###### 왜 넣지?
import warnings
from scipy import stats
warnings.filterwarnings('ignore')

In [ ]:
from collections import Counter

# Data

In [ ]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
df = getDF('/content/drive/MyDrive/Colab Notebooks/project/data/Arts_Crafts_and_Sewing.json.gz')

In [ ]:
metadf = getDF('/content/drive/MyDrive/Colab Notebooks/project/data/meta_Arts_Crafts_and_Sewing.json.gz')

In [ ]:
total = pd.merge(df, metadf, how='left', on='asin')

# Category

In [ ]:
crop = pd.DataFrame(total['category'])

In [ ]:
# null값을 찾아줍니다.
crop.isnull().sum()

category    8668
dtype: int64

In [ ]:
# 꼭 변수를 저장하자...
crop = crop.fillna('n')

In [ ]:
crop.isnull().sum()

category    0
dtype: int64

In [ ]:
# 데이터프레임을 리스트로 바꿉니다.
crop = crop['category'].tolist()

In [ ]:
type(crop[2])

str

In [ ]:
len(crop)

2911389

## unique category

In [ ]:
# 리스트에서 unique한 값만 뽑아줍니다.
crop_unique = [list(x) for x in set(tuple(x) for x in crop)]

In [ ]:
# 왜 할 때마다 달라지는지... 왜...
crop_unique[:2]

[['Arts, Crafts & Sewing',
  'Beading & Jewelry Making',
  'Beads & Bead Assortments',
  '100% Brand New And High Quality',
  'Material:Alloy&Rhinestone',
  'Color: Show as the picture (sent in random)',
  'Include packages: 1pair x Earrings'],
 ['Arts, Crafts & Sewing',
  'Beading & Jewelry Making',
  'Beads & Bead Assortments',
  '44 mm Height - 15 mm Width',
  '12 gms']]

In [ ]:
# 원래 4153개였는데 왜 하나가 늘어났는지... 'o'는 어디서 나왔는지...
len(crop_unique)

4154

In [ ]:
len(crop_unique[0])

7

In [ ]:
# 2번째 카테고리만 뽑아줍니다.
crop_one = []
for x in range(len(crop_unique)):
    if len(crop_unique[x]) >= 2:
        crop_one.append(crop_unique[x][1])
    else:
        crop_one.append('no')

In [ ]:
# unique한 2번째 카테고리 개수를 셉니다. 여러번 해도 이건 똑같으니 괜찮겠지.
Counter(crop_one)

Counter({'Beading & Jewelry Making': 2099,
         'Crafting': 231,
         'Fabric': 11,
         'Fabric Decorating': 7,
         'Knitting & Crochet': 31,
         'Needlework': 33,
         'Organization, Storage & Transport': 38,
         'Painting, Drawing & Art Supplies': 135,
         'Printmaking': 19,
         'Scrapbooking & Stamping': 40,
         'Sewing': 1508,
         'no': 2})

In [ ]:
# 이건 그냥 unique한 카테고리만 모은 리스트.
crop_one_u = []
for x in crop_one:
    if x not in crop_one_u:
        crop_one_u.append(x)
crop_one_u     # 'o'는 뭔지 모르겠다.

['Beading & Jewelry Making',
 'Sewing',
 'Crafting',
 'Knitting & Crochet',
 'Printmaking',
 'Scrapbooking & Stamping',
 'Needlework',
 'Painting, Drawing & Art Supplies',
 'Fabric',
 'Organization, Storage & Transport',
 'Fabric Decorating',
 'no']

## 전체 카테고리

In [ ]:
crop[:10]

['no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no']

In [ ]:
crop[9000:9003]

[['Arts, Crafts & Sewing',
  'Painting, Drawing & Art Supplies',
  'Painting',
  'Paints'],
 ['Arts, Crafts & Sewing',
  'Painting, Drawing & Art Supplies',
  'Painting',
  'Paints'],
 ['Arts, Crafts & Sewing',
  'Painting, Drawing & Art Supplies',
  'Painting',
  'Paints']]

In [ ]:
crop_total = []
for x in range(len(crop)):
    if len(crop[x]) >= 2:
        crop_total.append(crop[x][1])
    else:
        crop_total.append('o')

In [ ]:
crop_total[9000:9003]

['Painting, Drawing & Art Supplies',
 'Painting, Drawing & Art Supplies',
 'Painting, Drawing & Art Supplies']

In [ ]:
len(crop_total)

2911389

In [ ]:
# 원본 데이터셋에 'cat' 컬럼 추가
total['cat'] = crop_total

In [ ]:
total[9000:9003]

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image_x,category,tech1,description,fit,title,also_buy,image_y,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,cat
9000,4.0,NaN,True,"04 11, 2017",A17Q5SJG2VOILV,B00004THXH,{'Style Name:': ' Field Box 12 Half Pans'},Dsolo,The product is good and the price is great. I ...,Four Stars,1491868800,NaN,"[Arts, Crafts & Sewing, Painting, Drawing & Ar...",,[This Winsor & Newton pocket-sized lightweight...,,Winsor &amp; Newton Cotman Water Colour Paint ...,"[B004M59O4C, B018QCQONA, B015ZBIME4, B00AX31TZ...",[https://images-na.ssl-images-amazon.com/image...,,Winsor &amp; Newton,[Pocket-sized lightweight box is ideal for the...,"[>#12,020 in Arts, Crafts & Sewing (See Top 10...","[B000M53R62, B001OVG40E, B000N85WVQ, B001M6VMT...",{},Amazon Home,,,$16.50,"Painting, Drawing & Art Supplies"
9001,4.0,NaN,True,"04 11, 2017",A17Q5SJG2VOILV,B00004THXH,{'Style Name:': ' Field Box 12 Half Pans'},Dsolo,The product is good and the price is great. I ...,Four Stars,1491868800,NaN,"[Arts, Crafts & Sewing, Painting, Drawing & Ar...",,[This Winsor & Newton pocket-sized lightweight...,,Winsor &amp; Newton Cotman Water Colour Paint ...,"[B004M59O4C, B018QCQONA, B015ZBIME4, B00AX31TZ...",[https://images-na.ssl-images-amazon.com/image...,,Winsor &amp; Newton,[Pocket-sized lightweight box is ideal for the...,"[>#12,020 in Arts, Crafts & Sewing (See Top 10...","[B000M53R62, B001OVG40E, B000N85WVQ, B001M6VMT...",{},Amazon Home,,,$16.50,"Painting, Drawing & Art Supplies"
9002,5.0,3,True,"04 11, 2017",A397RJMNAF5SHO,B00004THXH,{'Style Name:': ' Sketchers Pocket Box'},Diego Acosta,"I'm a beginner to the watercolor medium, as a ...",Great Beginner start up,1491868800,[https://images-na.ssl-images-amazon.com/image...,"[Arts, Crafts & Sewing, Painting, Drawing & Ar...",,[This Winsor & Newton pocket-sized lightweight...,,Winsor &amp; Newton Cotman Water Colour Paint ...,"[B004M59O4C, B018QCQONA, B015ZBIME4, B00AX31TZ...",[https://images-na.ssl-images-amazon.com/image...,,Winsor &amp; Newton,[Pocket-sized lightweight box is ideal for the...,"[>#12,020 in Arts, Crafts & Sewing (See Top 10...","[B000M53R62, B001OVG40E, B000N85WVQ, B001M6VMT...",{},Amazon Home,,,$16.50,"Painting, Drawing & Art Supplies"


# Price

In [ ]:
# price 컬럼에서 na값 제거
total2 = total.dropna(subset=['price'])
total2['price'] = total2['price'].str.replace('$', '')

In [ ]:
drop_values = ['a', 'A', '-', '<']
total3 = total2[~total2.price.str.contains('|'.join(drop_values))]
total3['price'] = total3['price'].str.replace(',', '')
total3 = total3.dropna(subset=['price'])

In [ ]:
# price 전처리를 하고 나니 공백인 cell이 있어서 공백을 0으로 대체
total3['price'].replace('', '0')

2041        6.41
2042        6.41
2043        6.41
2044        6.41
2045           0
           ...  
2911384    10.99
2911385    10.99
2911386    10.99
2911387    10.99
2911388    10.99
Name: price, Length: 2855995, dtype: object

In [ ]:
# 전처리 끝난 문자열들을 실수타입으로 변환
total3['price'] = total3['price'].replace('', '0').astype('float')

In [ ]:
total3 = total3.drop(total3[total3['price'] == 0].index)      # inplace=False 그대로 둬야 함.

In [ ]:
total3

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image_x,category,tech1,description,fit,title,also_buy,image_y,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price
2041,5.0,NaN,True,"03 13, 2017",A3DKYRF9YGF6A6,6665560953,NaN,Amazon Customer,"Great quality product, and top notch service f...",Five Stars,1489363200,NaN,"[Arts, Crafts & Sewing, Sewing, Trim & Embelli...",,[The patch features the sweaty masculine hands...,,You Son of a Bitch! 1987 Embroidered Patch,"[6665562336, B072JVTJL5, B073SF9S8Y, B07H2V9DS...",[https://images-na.ssl-images-amazon.com/image...,,Honchosfx,[You son of a bitch patch - exclusive to Honch...,"122,174 in Clothing, Shoes & Jewelry (","[B073SF9S8Y, B07JMVMJT7, B0773TCDKV, B074MFG4C...",{},"<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",6.41
2042,5.0,NaN,True,"02 6, 2017",A1I2TJ3PQ4WHK1,6665560953,NaN,Eric Breneman,"details on this patch are very good, def a fav...",don't push too many pencils,1486339200,NaN,"[Arts, Crafts & Sewing, Sewing, Trim & Embelli...",,[The patch features the sweaty masculine hands...,,You Son of a Bitch! 1987 Embroidered Patch,"[6665562336, B072JVTJL5, B073SF9S8Y, B07H2V9DS...",[https://images-na.ssl-images-amazon.com/image...,,Honchosfx,[You son of a bitch patch - exclusive to Honch...,"122,174 in Clothing, Shoes & Jewelry (","[B073SF9S8Y, B07JMVMJT7, B0773TCDKV, B074MFG4C...",{},"<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",6.41
2043,5.0,NaN,True,"12 23, 2016",A1ZDJ2BYZXZHLQ,6665560953,NaN,Kirkja,Bought these as a gifts for friends and really...,Great deal if you buy in bulk!,1482451200,NaN,"[Arts, Crafts & Sewing, Sewing, Trim & Embelli...",,[The patch features the sweaty masculine hands...,,You Son of a Bitch! 1987 Embroidered Patch,"[6665562336, B072JVTJL5, B073SF9S8Y, B07H2V9DS...",[https://images-na.ssl-images-amazon.com/image...,,Honchosfx,[You son of a bitch patch - exclusive to Honch...,"122,174 in Clothing, Shoes & Jewelry (","[B073SF9S8Y, B07JMVMJT7, B0773TCDKV, B074MFG4C...",{},"<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",6.41
2044,5.0,NaN,True,"12 14, 2016",A1RW3PF3UTARAV,6665560953,NaN,Brian K.,The photo's really didn't do this patch justic...,It's made extremely well and looks GREAT! I've...,1481673600,NaN,"[Arts, Crafts & Sewing, Sewing, Trim & Embelli...",,[The patch features the sweaty masculine hands...,,You Son of a Bitch! 1987 Embroidered Patch,"[6665562336, B072JVTJL5, B073SF9S8Y, B07H2V9DS...",[https://images-na.ssl-images-amazon.com/image...,,Honchosfx,[You son of a bitch patch - exclusive to Honch...,"122,174 in Clothing, Shoes & Jewelry (","[B073SF9S8Y, B07JMVMJT7, B0773TCDKV, B074MFG4C...",{},"<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",6.41
2046,4.0,69,True,"03 4, 2010",A3BQSWC3801JNI,8862933177,{'Format:': ' Hardcover'},Stacy A. Jones,I have to be honest - I already have quite a c...,"Solid buy, few small gripes about page layout",1267660800,NaN,"[Arts, Crafts & Sewing, Crafting, Paper & Pape...","class=""a-keyvalue prodDetTable"" role=""present...",[Build your very own film archive. Fully embos...,,"Moleskine Passion Journal - Film, Large, Hard ...","[1926892801, 8862933193, 8862933118]",[https://images-na.ssl-images-amazon.com/image...,,Moleskine,[Used Book in Good Condition],"[>#229,441 in Office Products (See top 100), >...","[8862933193, B001KN2B08, 1948713047, 886293315...",{},Office Products,,"November 3, 2009",137.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2911384,3.0,2,True,"12 5, 2016",A3DBSNSXS3HLW2,B01HJHR2PM,NaN,Amazon Customer,The pens took weeks to arrive and ran out of i...,... took weeks to arrive and ran out of ink pr...,148089600

In [ ]:
# 가격 최대/최소
price_sort = total3['price'].unique()
print(price_sort.max(), price_sort.min())

14845.95 0.01


In [ ]:
# 가격 통계
round(total3['price'].describe(), 3)

count    2286798.000
mean          16.111
std           42.178
min            0.010
25%            6.680
50%            9.950
75%           15.990
max        14845.950
Name: price, dtype: float64

In [ ]:
total3[total3['price'] == 14845.95]['title'].tolist()

['CLEARSNAP Clear Scraps Border Stencils, 3 x 12, Waves']

In [ ]:
price_desc = np.sort(total3['price'])[::-1]
price_desc[:5]

array([14845.95, 10995.  ,  9999.  ,  9999.  ,  9999.  ])

In [ ]:
total3[total3['price'] == 10995.00]

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image_x,category,tech1,description,fit,title,also_buy,image_y,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price
2595876,5.0,NaN,False,"08 31, 2015",A23ATE2DRL0EWU,B00OXFHNF8,NaN,SignSmoker,We have had one for years and love it! Great f...,We have had one for years and love it! Great f...,1440979200,NaN,"[Arts, Crafts & Sewing, Crafting, Craft Suppli...",,"[EGX-400 NO RETUNS Max. cutting area 16""(X) x ...",,EGX-400 Computer Engraver,[],[https://images-na.ssl-images-amazon.com/image...,,Roland DGA,"[EGX-400 COMPUTER ENGRAVER, This Machine is no...","[>#2,603,221 in Arts, Crafts & Sewing (See Top...","[B07KYH6BTK, B00I0DGX0U, B07HP4LMN2, B00BKWOSL...",{},"Arts, Crafts & Sewing",,,10995.0


In [ ]:
def price_selector(a=0, b=0):
  if a > 0 and b > 0:
    review_df_x = total3[(total3.price > a) & (total3.price < b)]
  elif b == 0:
    review_df_x = total3[total3.price > a]
  elif a == 0:
    review_df_x = total3[total3.price < b]

  return review_df_x

In [ ]:
review_ex = price_selector(0, 1)
review_ex

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image_x,category,tech1,description,fit,title,also_buy,image_y,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price
2535,5.0,NaN,False,"08 26, 2014",ANHBDJMK3M5RB,B000003OP3,{'Format:': ' Audio CD'},Vernon L Hershberger,ok,Five Stars,1409011200,NaN,"[Arts, Crafts & Sewing, Needlework, Cross-Stit...",,[Carl Butler and Pearl were an American countr...,,Don't Let Me Cross Over,"[B01LTHP1GA, B0000282TN, B07G1SFVT1, B000C3GWG...",[],,Carl & Pearl Butler,[],"2,210,579 in CDs & Vinyl (","[B0000282TN, B01LTHP1GA, B07G1SFVT1, B000003OO...","{'Audio CD': '(May 6, 1997)', 'Number of Discs...","<img src=""https://images-na.ssl-images-amazon....",,,0.70
2536,5.0,NaN,True,"08 6, 2014",A14VKCD3KUFIK9,B000003OP3,{'Format:': ' Audio CD'},LINDA L LEWIS,My husband loves this,Five Stars,1407283200,NaN,"[Arts, Crafts & Sewing, Needlework, Cross-Stit...",,[Carl Butler and Pearl were an American countr...,,Don't Let Me Cross Over,"[B01LTHP1GA, B0000282TN, B07G1SFVT1, B000C3GWG...",[],,Carl & Pearl Butler,[],"2,210,579 in CDs & Vinyl (","[B0000282TN, B01LTHP1GA, B07G1SFVT1, B000003OO...","{'Audio CD': '(May 6, 1997)', 'Number of Discs...","<img src=""https://images-na.ssl-images-amazon....",,,0.70
2537,2.0,NaN,True,"12 13, 2013",A38W15EVNZKC78,B000003OP3,{'Format:': ' Audio CD'},Ann,I was disappointed in Don't Let Me Cross Over....,Disappointed in Don't Let Me Cross Over.,1386892800,NaN,"[Arts, Crafts & Sewing, Needlework, Cross-Stit...",,[Carl Butler and Pearl were an American countr...,,Don't Let Me Cross Over,"[B01LTHP1GA, B0000282TN, B07G1SFVT1, B000C3GWG...",[],,Carl & Pearl Butler,[],"2,210,579 in CDs & Vinyl (","[B0000282TN, B01LTHP1GA, B07G1SFVT1, B000003OO...","{'Audio CD': '(May 6, 1997)', 'Number of Discs...","<img src=""https://images-na.ssl-images-amazon....",,,0.70
2538,5.0,NaN,True,"05 9, 2013",A394T02VQOKZJ2,B000003OP3,{'Format:': ' Audio CD'},Dudley K Sluder,I love duets from the 60's and this is right u...,"50's and 60""s country at its best.",1368057600,NaN,"[Arts, Crafts & Sewing, Needlework, Cross-Stit...",,[Carl Butler and Pearl were an American countr...,,Don't Let Me Cross Over,"[B01LTHP1GA, B0000282TN, B07G1SFVT1, B000C3GWG...",[],,Carl & Pearl Butler,[],"2,210,579 in CDs & Vinyl (","[B0000282TN, B01LTHP1GA, B07G1SFVT1, B000003OO...","{'Audio CD': '(May 6, 1997)', 'Number of Discs...","<img src=""https://images-na.ssl-images-amazon....",,,0.70
2539,5.0,NaN,True,"02 10, 2013",A1QVJLIANHN54P,B000003OP3,{'Format:': ' Audio CD'},Raymond E. Marshall,I greatly enjoy this classic country music and...,Great,1360454400,NaN,"[Arts, Crafts & Sewing, Needlework, Cross-Stit...",,[Carl Butler and Pearl were an American countr...,,Don't Let Me Cross Over,"[B01LTHP1GA, B0000282TN, B07G1SFVT1, B000C3GWG...",[],,Carl & Pearl Butler,[],"2,210,579 in CDs & Vinyl (","[B0000282TN, B01LTHP1GA, B07G1SFVT1, B000003OO...","{'Audio CD': '(May 6, 1997)', 'Number of Discs...","<img src=""https://images-na.ssl-images-amazon....",,,0.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899904,5.0,NaN,True,"12 25, 2016",A2WLAJYAP9P995,B01GM7Y9TC,NaN,SandyS,excellent product AS PROMISED,Five Stars,1482624000,NaN,"[Arts, Crafts & Sewing, Fabric]",,[Nickel-Copper Rip-Stop (Anti-Rip) RF Shieldin...,,Faraday Defense RFID Shielding Nickel Copper R...,[],[https://images-na.ssl-images-amazon.com/image...,,Faraday Defense,[High Performance Copper Nickel Anti-Rip Rip-S...,"[>#221,979 in Electronics (See Top 100 in Elec...","[B01M294MGK, B01HPFAV6K, B075WCRP1V, B01LBFCJ7...",{},Computers,,"June 28, 2016",0.37
2903707,5.0,NaN,True,"04 2, 2018",A325V74R30F56O,B01GYES8F4,{'Size:': ' 39x7mm'},T. Lewis,These make great shoelace tags that don't rust...,Great quality and size. I've ordered several t...,1522627200,NaN,"[Arts, Crafts & Sewing, Beading & Jewelry Maki...",,[<b>Descripti

In [ ]:
review_ex['title'][293707]

KeyError: ignored